In [ ]:
# Standard library
import os
import urllib
import zipfile
from tempfile import NamedTemporaryFile
import shutil
import datetime as dt
from ftplib import FTP
from time import sleep

# Installed packages
from viresclient import SwarmRequest
import pandas as pd
import cdflib
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Autoreload changes to module utils.py
%load_ext autoreload
%autoreload 2
import utils

# Directory to use for outputs
os.makedirs("output", exist_ok=True)

# Indicate primary package versions to help debugging
%load_ext watermark
%watermark -i -v -p viresclient,cdflib,eoxmagmod,pandas,xarray,matplotlib,cartopy

# 1. Field intensity overview

In [ ]:
# Identify recent 7-day time window to use
today = dt.datetime.combine(dt.date.today(), dt.time())
three_days_ago = today - dt.timedelta(days=3)
ten_days_ago = today - dt.timedelta(days=10)
start, end = ten_days_ago, three_days_ago
start, end

In [ ]:
# Fetch data
request = SwarmRequest()
request.set_collection("SW_OPER_MAGA_LR_1B")
request.set_products(
    measurements=["F"],
    sampling_step="PT1M"
)
request.set_range_filter("Flags_F", 0, 1)
data = request.get_between(start, end)
ds = data.as_xarray()
ds

In [ ]:
# Identify percent of data that is available
percent_nominal = 100*len(ds["Timestamp"])/10080
percent_nominal

In [ ]:
# Build figure to plot onto
fig = plt.figure(figsize=(10, 5))
ax_Moll = plt.subplot2grid(
    (1, 1), (0, 0), colspan=1,
    projection=ccrs.Mollweide()
)
ax_Moll.add_feature(cfeature.COASTLINE, edgecolor='black', alpha=0.5)

# Draw on the measurements
ds.plot.scatter(
    "Longitude", "Latitude", "F",
    ax=ax_Moll, transform=ccrs.PlateCarree(),
    s=10, alpha=0.4, marker=".",
    cmap="jet", norm=plt.Normalize(20e3, 50e3)
)

# Evaluate model and draw contours
coords = utils.grid(height=430)
IGRF = utils.eval_model(time=start,coords=coords)
utils.plot_contours(
    ax_Moll,
    coords[..., 1], coords[..., 0], IGRF["F"],
    draw_labels=False, linewidths=2, levels=20, alpha=0.5,
    cmap="jet", norm=plt.Normalize(20e3, 50e3)
)

# Prepare descriptive text
ID2NAME = {"A": "ALPHA", "B": "BRAVO", "C": "CHARLIE"}
spacecraft = ID2NAME.get(ds['Spacecraft'].values[0])
spacecraft = f"SWARM {spacecraft}"
altitude = int(round((ds["Radius"].values[0] - 6371200)/1e3, ndigits=-1))
quantity = "Magnetic field intensity (F) / nanoTesla"
sample_rate = "1 Minute"
description = f"""
Spacecraft:   {spacecraft} (~{altitude}km altitude)
Time period:  {start.strftime('%Y-%m-%d')} to {(end-dt.timedelta(days=1)).strftime('%Y-%m-%d')}
Quantity:     {quantity}
Downsampling: {sample_rate} (Flags_F: 0-1)
[ Dots: Measurements, Contours: Model ]
""".strip()
# Place text
fig.text(0, 1.05, description, va="top", fontfamily="monospace", fontsize=14);
fig.savefig("output/intensity.png", bbox_inches="tight", dpi=150)

# 2. Swarm Dst-like index (MMA Fast-track)

### Fetch the most recent available MMA_SHA_2F file

In [ ]:
def fetch_zipped_file(url, file_name):
    """Fetch a given file from within an online zip file"""
    output_file = NamedTemporaryFile()
    zip_file, _ = urllib.request.urlretrieve(url)
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        with zip_ref.open(file_name) as f:
            shutil.copyfileobj(f, output_file)
            output_file.seek(0)
    return output_file

def fetch_mma_file():
    """Fetch most recent available MMA_SHA_2F"""
    # List MMA_SHA_2F product files available for current year
    host = "swarm-diss.eo.esa.int"
    file_root = "/Level2longterm/MMA/SW_OPER_MMA_SHA_2F_" + \
                f"{dt.date.today().year}0101T000000_*"
    print(f"Connecting to: {host}")
    print(f"Looking for: {file_root}")
    with FTP(host) as ftp:
        ftp.login()
        files = ftp.nlst(file_root)
    # Identify most recent one
    files.sort()
    matched_file = files[-1]
    file_url = "ftp://" + host + matched_file
    file_name = matched_file.split("/")[-1].split(".ZIP")[0] + ".cdf"
    print(f"Using: {file_url}")
    print(f"Attempting to extract: {file_name}")
    return fetch_zipped_file(file_url, file_name)

def try_x_times(fn, x=3, delay_factor=10):
    """Attempt function, fn, x times, with increasing delays"""
    for attempt in range(x):
        try:
            outputs = fn()
        except Exception as e:
            if attempt < x - 1:
                delay = (attempt + 1) * delay_factor
                print(f"Failed ({e}). Trying again in {delay}s...")
                sleep(delay)
                continue
            else:
                raise
        return outputs

mma_file = try_x_times(fetch_mma_file, x=3, delay_factor=30)

In [ ]:
# Open the file and show contents
mma_data = cdflib.CDF(mma_file.name)
mma_data.cdf_info()

In [ ]:
time = mma_data.varget('t_gh')[0]
gh10 = mma_data.varget('qs_geo')[:, :, 0][0]
time, gh10

In [ ]:
# Load as a basic dataframe
CDF_EPOCH_1970 = 62167219200000.0
time = mma_data.varget('t_gh')[0]
df_mma = pd.DataFrame.from_dict({
    "Timestamp": pd.to_datetime((time - CDF_EPOCH_1970)/1e3, unit="s"),
    "gh_10": 
        mma_data.varget('gh_geo')[:, :, 0][0],
    "qs_10":
        mma_data.varget('qs_geo')[:, :, 0][0]
}
).set_index("Timestamp")
df_mma.head()

In [ ]:
fig, axes = plt.subplots(nrows=1, sharex=True, figsize=(10,5))
ax = axes

df_month = df_mma[(df_mma.index[-1] - dt.timedelta(days=28)):]
# df_week  = df_mma[(df_mma.index[-1] - dt.timedelta(days=7)):]
df_month["qs_10"].plot(color="firebrick", ax=ax)
df_month["gh_10"].plot(color="mediumblue", ax=ax)
ax.set_xlim((df_month.index[0], df_month.index[-1]))
ax.set_ylim((-20, 100))
## why isn't this working?...
# ax.xaxis.set_major_formatter(DateFormatter("%Y-%m-%d\n%H:%M:%S"))
ax.grid()
ax.set_ylabel("MMA_F (q$^1_0$, g$^1_0$) [nT]");
description = f"""
Swarm-derived magnetospheric field estimate (like Dst index)
Product: MMA_SHA_2F (fast-track)
Time period:  {df_month.index[0].strftime('%Y-%m-%d')} to {df_month.index[-1].strftime('%Y-%m-%d')}
""".strip()
# Place text
fig.text(0, 0.95, description, va="bottom", fontfamily="monospace", fontsize=14);
ax.text((df_mma.index[-1] - dt.timedelta(days=7)), 90, "← past month ", ha="right", fontsize=14)
ax.text((df_mma.index[-1]), 90, "← past week ", ha="right", fontsize=14)
ax.text(df_month.index[0], 83, " External (primary) field", color="firebrick", fontsize=14)
ax.text(df_month.index[0], 74, " Internal (induced) field", color="mediumblue", fontsize=14)
ax.axvline(x=df_mma.index[-1] - dt.timedelta(days=7), color="black", ls="--", alpha=0.5)
ax.axhline(y=0, color="black", alpha=0.5)
fig.savefig("output/mma-f.png", bbox_inches="tight", dpi=150)